# Inverted Pendulum using DQN

When moving on from discrete environments, such as grid world or black jack, the amount of state in a continous environment renders traditional tabular methods (Monte Carlo and TD learning) intractable (at least without any modification to the problem formulation). DQN method addresses this problem by using a value function approximator, essentially replacing the Q-table of explicit state-action value with a parameterized function $q^\pi_{\vec w}(s,a,\vec w)$. The vector $\vec w$ stores the weights or parameters of this approximating function.

The motivation of using value function approximator is clear, but where does it fit into the RL picture?
- Value function approximator is a direct mapping between the states (features) to the state-action value $Q(s,\cdot)$ 
- As such, the approximator function fits in the policy evaluation step of the RL training process. We train this function by minimizing the loss between the approximated value function and the actual value function $ s.t. \enspace q^\pi_{\vec w}(s,a,\vec w) \approx q^*$
- However, the true action value function $q^*$ is unknown and must be learnt from the interaction between the agent and its environment. We must employ Q-learning to approximate the true action value function, toward which we will update the parameterized value function $q^\pi_{\vec w}(s,a,\vec w)$. The update is as follow

**The DQN algorithm**
1. From a state $s$, take action $a$ using $\epsilon$-greedy policy, observe the reward $r$ and next state $s'$
2. Store the transition tuple $(s, a, r, s')$ in a replay memory $\mathcal{D}$. This replay memory is a buffer that stores the most recent transition tuples
3. From this replay memory, sample batches of $(s_t, a_t, r_{t+1}, s_{t+1})$ randomly for training the Q-function approximator
4. Adjust the parameters $\vec w$ by optimizing the mean squared error (MSE) between the Q network and the Q-learning target using Q-learning
$$ \mathcal{L}_i (\vec w_i) = \mathbb{E}_{(s,a,r,s') \sim \mathcal{D}} \bigg[ \Big(r + \gamma \max_{a'}Q(s',a',\vec w^-_i) - Q(s,a,\vec w_i) \Big)^2\bigg]$$ 

When applying stochastic gradient descent, the update at every step is reduced as follows (the math found in the GoodNote notebook):

$$\Delta \vec w = \alpha \Big(r + \gamma \max_{a'} \underbrace{Q(s', a', \vec{w}_i^-)}_{\text{target}} - \underbrace{Q(s,a,\vec w_i)}_{\text{behaviour}} \Big) \vec x(s,a)$$


**Notes:** 

There are two mechanisms introduced in this paper that allow for stable training, one is experience replay, and the other is training two separate methods.
- **Experience replay** - this decorrelates the experience tuples (s,a,r,s') thus follow the i.i.d assumption for training a neural network
- **Two Q-networks**, a target network and a behaviour network. The parameters $\vec w'$ of the target network $Q(s,a,\vec w')$ are only updated infrequently to maintain a stable target. The parameters $\vec w$ of the behaviour network $Q(s,a,\vec w)$ are updated every policy improvement iteration to move toward this target network. The frequency of updating the target network is annother hyperparameter to tune. 


## Question to self
1. What does it mean to apply stochastic gradient descent to simplify the loss function? What are the implications?
- Every policy evaluation iteration, a random sample, or batch of samples, is used for updating the state-action value instead of every single experience stored in the replay buffer. Recall that the random sampling from the replay buffer de-correlates the experience and enforces the i.i.d. assumption
- SGD is used on a small batch of sample to reduce the variance inherent in the different experience in the replay buffer. Despite the noise and bias of this method, the performance of the Q networks improves.


</br>

2. Would it be beneficial to formulate the Q network such that the input is the state vector and the outputs are the Q-values associated with each possible action from the state. As such, one can just take the max value from the target Q network and index into the behaviour network for the corresponding action a?
- That is the common approach for approximating the action value function, outputting Q(s) for all a, then grabbing the maximum value
 
</br>

3. At first do I have to run the agent several times using the initial $\epsilon$-greedy policy to fill the memory buffer with training samples?
- Yes, and then pop in new experience at the end, push the oldest experience in the replay buffer out.

</br>

4. How often does the behaviour target network updated?

- At every step in an episode of the training process, the behaviour network is used for two different purposes:
    - Firstly, the behaviour network governs the trajectory of the agent at every step. 
    - Secondly, the behaviour Q network is used to predict (or infer) the action value for the <s,a> pairs drawn from a batch of experience (s,a,r,s'). These value is compared to the discounted maximum action value at state s', $\max Q(s',\cdot)$, according to the **target network**. 

- The behaviour network prediction of $Q(s,a)$ and the target network's label output of $r+\gamma \max Q(s',a')$ are used to compute a loss function. The gradient of this loss function is used for adjusting the parameters of the behaviour network.
- After the agent has taken a number of step, the parameters (or weights) or the behaviour network is loaded onto the target network.

</br>

5. How can I scale this environment up to have continuous action instead of just left/right?
- To my understanding, actuating the cart to the left or right at similar output level (voltage) at instantaneous moments throughout the control process is good enough for modulating the speed of the cart. Think of PWMs where the voltage is just switch at a certain duty cycle to simulate different voltage level.
- It is possible to have continuous action space. Since the Q network output is the state-action value Q(s,a) at each state s and action a, we know how good each action is in a state. The Q values can then be transformed into a proportional constant to control how much to move left or right (instead of just a binary left/right). This essentially means that the Q-value output of this network is not simply used for selecting the maximizing action at each state but also for governing the magnitude of cart actuation.


In [ ]:
import random                       # To draw a random batch of samples form the replay buffer
import gymnasium as gym
import torch
from torch import nn
import torch.optim as optim
import numpy as np

from collections import deque       # replay buffer is a double ended queue that allows elements to be added either to the end or the start

In [ ]:
# Simulate several episodes of the cart pole environment (random policy on a human render mode)
env = gym.make("CartPole-v1", render_mode = "human")

state, _ = env.reset(); t = 1
for i in range(100):
    action = env.action_space.sample()
    next_state, reward, term, trunc, info = env.step(action)
    print(f"Time {t:2d}  |  s_t {state}  |  a_t {action:2d}  |  s_t+1 {next_state}  |  reward {reward:.2f}  |  terminated {term:2}  |  {info}")
    state = next_state; t += 1
    if term or trunc:
        break

In [ ]:
# Constant definitions
LR = 1e-3
BUFFER_SIZE = 5000                  # size of the experience replay buffer
MIN_REPLAY_SIZE = 1000              # The number of experience to sample before starting training
TARGET_UPDATE_FREQ = 1000           # the number of steps before updating the target network

GAMMA = 0.95                        # Discount factor for return calculation
EPSILON_START = 1.0                 # exploration rate in stochastic policy
EPSILON_END = 0.1                   # minimum exploration rate at the end of decay
EPSILON_DECAY = 5000                # epsilon decay rate

EPISODE_TRAIN = 5000              # number of episodes to train the Q-network
BATCH_SIZE = 32

In [ ]:
# Define Q-network architecture
class QNetwork(nn.Module):
    ''' A QNetwork class that initialize an MLP neural Q network with two hidden layers, each with 128 nodes '''
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)                       # The regression output are the state values of Q(s,a=left) and Q(s,a=right)

In [ ]:
# Dynamically define Q-network architecture

In [ ]:
# Initialize the environment and the Q network

env = gym.make("CartPole-v1")
state_space = env.observation_space.shape[0]        # State space of cart pos & vel, and pendulum angular pos and vel. Each component is a continuous value
action_space =  env.action_space.n                   # Action space of left or right

q_network = QNetwork(state_space,action_space)
target_network = QNetwork(state_space,action_space)
target_network.load_state_dict(q_network.state_dict())
target_network.eval()                               # target network is only used for evaluation/inference and not trained

optimizer = optim.SGD(q_network.parameters(), lr = LR)
replay_buffer = deque(maxlen=BUFFER_SIZE)

In [ ]:
# Fill replay buffer with some initial samples for training (random policy)
obs, _ = env.reset()
for _ in range(MIN_REPLAY_SIZE):
    action = env.action_space.sample()
    next_obs, reward, term, trunc, _ = env.step(action)
    done = term or trunc

    replay_buffer.append((obs, action, reward, next_obs, done))
    obs = next_obs if not done else env.reset()[0]

In [ ]:
# DQN Model Training loop
reward_history = np.zeros(EPISODE_TRAIN)
epsilon = EPSILON_START
step_count = 0
target_network_update_count = 0

for episode in range(EPISODE_TRAIN):
    obs, _ = env.reset()
    eps_rewards = 0
    

    while True:
        # Epsilon-greedy policy to select action
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():       # Doing inference so no need to track operations
                state_tensor = torch.FloatTensor(obs).unsqueeze(0)  # unsqueeze to fake the sample (batch) dimension
                q_values = q_network(state_tensor)
                action = q_values.argmax().item()                   # index of the maximum output (action)
        
        # Interact with the environment
        next_obs, reward, term, trunc, _ = env.step(action)
        done = term or trunc
        replay_buffer.append((obs, action, reward, next_obs, done))
        obs = next_obs
        eps_rewards += reward
        step_count += 1

        # Sample a batch and update the network
        if len(replay_buffer) >= BATCH_SIZE:
            batch = random.sample(replay_buffer, BATCH_SIZE)
            states, actions, rewards, next_states, dones = zip(*batch)

            states = torch.FloatTensor(states)                      # Shape of (BATCH_SIZE, 4). Data structure of [[p1 v1 theta1 dtheta1], [p2 v2 theta2 dtheta2], ...]
            actions = torch.LongTensor(actions).unsqueeze(1)        # Must unsqueeze to have shape (BATCH_SIZE, 1). From [a1, a2, ...] to [[a1], [a2], ... [a_BATCH_SIZE]]
            rewards = torch.FloatTensor(rewards).unsqueeze(1)
            next_states = torch.FloatTensor(next_states)
            dones = torch.FloatTensor(dones).unsqueeze(1)

            # Compute targets using target network Q(s',a',w_i^-)
            with torch.no_grad():
                target_q_values = target_network(next_states)       # Find a batch of Q(s',a',w_i^-) from the batch of next_states
                max_target_q_values = target_q_values.max(dim=1, keepdim=True)[0]
                targets = rewards + GAMMA * max_target_q_values * (1 - dones)       # (1-dones) to avoid over estimating the value of terminal states. The target should only be the reward in the terminal states

            # Compute the current Q values for the actions taken Q(s,a,w_i)
            q_values = q_network(states).gather(1, actions)         # obtain the q_values associated to the actual action taken in each sample

            # Update the parameters of the behaviour q_network
            ## Strategy 1 - Compute the loss and update the q_network toward the target greedy network
            loss = nn.MSELoss()(q_values, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            ## Strategy 2 - Use Q-learning to update the weights manually

             # Periodically update the target network by loading the weights from the behavior network
            if step_count % TARGET_UPDATE_FREQ == 0:
                target_network_update_count += 1
                target_network.load_state_dict(q_network.state_dict())

        if done:
            break
    
   
    # Decay epsilon
    epsilon = max(EPSILON_END, epsilon - (EPSILON_START - EPSILON_END)/EPSILON_DECAY)

    reward_history[episode] = eps_rewards
    print(f"Episode {episode:5d}: Total reward = {eps_rewards:5.1f}, Epsilon = {epsilon:.3f}, Step count = {step_count:5d}, Target update count = {target_network_update_count:3d}")
    print(f"Average reward {sum(reward_history[:episode])/(episode+1):.2f}")

env.close()

In [ ]:
def EMA_filter(reward: list, alpha):
    output = np.zeros(len(reward)+1)
    output[0] = reward[0]
    for idx, item in enumerate(reward):
        output[idx+1] = (1 - alpha) * output[idx] + alpha * item
    
    return output

In [ ]:
# Visualize the history of success rate
import matplotlib.pyplot as plt

episode = range(EPISODE_TRAIN)
filtered_reward_history = EMA_filter(reward_history, 0.1)

# Set figure size (width, height) in inches
plt.figure(figsize=(20, 6))

slice_idx = len(reward_history)
# Plot y1 vs x on the first subplot
plt.plot(episode[:slice_idx], reward_history[:slice_idx], color = "blue")
plt.plot(episode[:slice_idx], filtered_reward_history[:slice_idx], color = "red")
plt.title('Success Rate vs Traing Episode')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.legend(['Total reward', 'Filtered reward'])

# Adjust layout
plt.tight_layout()
plt.show()

## Save the trained model

In [ ]:
import os                                               # For saving models and training results
from datetime import datetime                           # For creating the directory of each training run
import json                                             # For storing training parameters during each run

# Generate a timestamped directory for the training run
timestamp = datetime.now().strftime("%y%m%d_%H%M%S")
BASE_DIR = os.getcwd()
OUTPUT_DIR = os.path.join(BASE_DIR,f"inv_pend_results/{timestamp}_{EPISODE_TRAIN}_{BATCH_SIZE}_{LR}_{GAMMA}_{TARGET_UPDATE_FREQ}_{EPSILON_DECAY}")
os.makedirs(OUTPUT_DIR, exist_ok=True)

torch.save({
    'model_state_dict': q_network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    
    'learning_rate': LR,
    'buffer_size': BUFFER_SIZE,
    'min_replay_size': MIN_REPLAY_SIZE,
    'target_update_freq': TARGET_UPDATE_FREQ,

    'gamma': GAMMA,
    'epsilon_start': EPSILON_START,
    'epsilon_end': EPSILON_END,
    'epsilon_decay': EPSILON_DECAY,

    'episode_train': EPISODE_TRAIN,
    'batch_size': BATCH_SIZE
}, os.path.join(OUTPUT_DIR,'q_network_checkpoint.pth'))

In [ ]:
checkpoint = torch.load("C:/Minh Nguyen/experiment/DRL/gym_exercise/inv_pend_results/250520_165708_5000_32_0.001_0.95_1000_5000/q_network_checkpoint.pth")
q_network_loaded = QNetwork(state_space,action_space)
q_network_loaded.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
print(checkpoint.keys())
print(checkpoint['optimizer_state_dict'].keys())

## Performance Visualization

Visualize the performance of the trained Q network in multiple simulations and observer the reward.

In [ ]:
env_test = gym.make("CartPole-v1", render_mode = "human")

# print(type(obs))
for episode in range(5):
    obs, _ = env_test.reset()
    done = False
    eps_reward = 0

    while not done:
        with torch.no_grad():       # Doing inference so no need to track operations
            state_tensor = torch.FloatTensor(obs).unsqueeze(0)  # unsqueeze to fake the sample (batch) dimension
            q_values = q_network(state_tensor)
            action = q_values.argmax().item()                   # index of the maximum output (action)
        
        next_obs, reward, term, trunc, _ = env_test.step(action)

        done = term or trunc
        eps_reward += reward
        obs = next_obs
    
    print(f"Reward from episode {episode:3d} is {eps_reward}")

Visualize the state vs time plot

In [ ]:
import matplotlib.pyplot as plt

obs, _ = env_test.reset()
done = False
obs_history = []; obs_history.append(obs)
action_history = []
reward_history = []

while not done:
    with torch.no_grad():       # Doing inference so no need to track operations
        state_tensor = torch.FloatTensor(obs).unsqueeze(0)  # unsqueeze to fake the sample (batch) dimension
        q_values = q_network(state_tensor)
        action = q_values.argmax().item()                   # index of the maximum output (action)
    
    next_obs, reward, term, trunc, _ = env_test.step(action)

    action_history.append(action)
    obs_history.append(next_obs)
    
    if not reward_history: reward_history.append(reward)
    else: reward_history.append(reward_history[-1] + reward)

    done = term or trunc
    obs = next_obs

# Convert obs_history to a NumPy array for easier slicing
obs_array = np.array(obs_history)

# Create subplots
fig, axs = plt.subplots(6, 1, figsize=(12, 24), sharex=True)

# 1. Plot cumulative reward history
axs[0].plot(reward_history, label='Cumulative Reward', color='green')
axs[0].set_ylabel('Cumulative Reward')
axs[0].legend()
axs[0].grid(True)

# 2. Plot action history
axs[1].plot(action_history, label='Actions Taken', color='blue')
axs[1].set_ylabel('Action')
axs[1].legend()
axs[1].grid(True)

# 3. Plot each CartPole state variable over time
state_labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']
for i in range(4):
    axs[i+2].plot(obs_array[:, i], label=state_labels[i])
    axs[i+2].set_ylabel('State Values')
    axs[i+2].set_xlabel('Timestep')
    axs[i+2].legend()
    axs[i+2].grid(True)

plt.tight_layout()
plt.show()


# Inverted Pendulum using double DQN (DDQN)

In [ ]:
import random                       # To draw a random batch of samples form the replay buffer
import gymnasium as gym             # To create the inverted pendulum environment
import torch                        
from torch import nn
import torch.optim as optim
import numpy as np
import re

from collections import deque       # replay buffer is a double ended queue that allows elements to be added either to the end or the start

In [ ]:
class QNet_MLP(nn.Module):
    ''' A QNetwork class that dynamically initialize an MLP Q Net from a list specifying the number of nodes in each hidden layer (e.g. [64,32])'''
    def __init__(self,input_dim,output_dim,hidden_layer = [64,32]):
        super().__init__()

        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.output_dim = output_dim

        for size in hidden_layer:
            self.layers.append(nn.Linear(self.input_dim, size))
            self.input_dim = size
        
        self.layers.append(nn.Linear(self.input_dim,self.output_dim))
    
    def forward(self, input_data):
        for layer in self.layers[:-1]:
            input_data = torch.relu(layer(input_data))
        return self.layers[-1](input_data)

In [ ]:
class DDQN_experiment():
    def __init__(self, model_name: str,      # "DQN_MLP_v0" or "DQN_MLP_v1"
                 model_registry, 
                 lr: float, 
                 buffer_size: int, 
                 target_update_freq: int, 
                 gamma: float, 
                 eps_start: float, 
                 eps_decay: int, batch_size: int):
        ''' Defining hyperparameters in the experiment '''
        self.model_name = model_name                                        # Full name of the model
        self.model_class = model_name[self.model_name]['class']             # The model class "QNet_MLP" or "QNet_test"
        self.model_config = model_registry[self.model_name]['config']       # List of nodes in each hidden layer
        match = re.search(r'v\d+',self.model_name)
        self.model_id = match.group(0) if match else 404                    # Extract the "v0" or "v1" out of model name for abbreviation

        # Hyperparameters of the experiment
        self.lr = lr
        self.buffer_size = buffer_size
        self.target_update_freq = target_update_freq
        self.gamma = gamma
        self.eps = eps_start
        self.eps_decay = eps_decay
        self.batch_size = batch_size

        self.env = gym.make("CartPole-v1")
        obs_space = self.env.observation_space.shape[0]
        act_space = self.env.action_space.n
        self.Q_net = QNet_MLP(obs_space, act_space, self.model_config)
        self.target_net = 


    def create_directory(self):
        ''' Function that creates directory to save model state_dict, architecture, training configuration, and history

        Parameters: 
        ------------
        (hyperparameters for differentiating between different directory)
        
        lr : float
            the learning rate to optimize the Q network
        gamma : float 
            the discount rate in Q learning
        epsilon_decay : integer
            the amount of episode over which the exploratory rate (epsilon) decays
        batch_size : integer
            number of experience drawn from replay buffer to train the behaviour network
        buffer_size : integer
            the number of samples in the replay buffer at a time
        target_udpate_freq : integer
            the amount of step count during the training process before updating the target Q net (loading the parameters of the behaviour net onto the target Q Net)
        

        Returns
        ------------
        name_codified : str
            the shortened name for the current experiment 
        hyperparameters_codified : str
            the shortened string of hyperparameter configuration
        OUTPUT_DIR : path
            the directory to which the training results and model (state_dict and architecture) will be saved
        '''
        timestamp = datetime.now().strftime("%y%m%d_%H%M")

        RESULT_FOLDER = "inv_pend_DDQN_results"
        BASE_DIR = os.getcwd()
        RESULT_DIR = os.path.join(BASE_DIR, RESULT_FOLDER)
        os.makedirs(RESULT_DIR, exist_ok=True)      # Create the directory if one does not already exist

        # Find the trial # of the latest run
        existing_runs = [d for d in os.listdir(RESULT_DIR) if os.path.isdir(os.path.join(RESULT_DIR,d))]
        run_numbers = [int(re.search(r'run_(\d{5})',d).group(1)) for d in existing_runs if re.match(r'run_\d{5}',d)]
        trial_number = max(run_numbers,default=-1)+1

        # Create a folder for the run
        name_codified = f"run_{trial_number:05d}"
        OUTPUT_DIR = os.path.join(RESULT_DIR,name_codified)
        os.makedirs(OUTPUT_DIR, exist_ok=True)      # Create the directory

        # Append the mapping from run # to hyperparameter configuration in a JSON file inside RESULT_DIR
        trial_to_param_path = os.path.join(RESULT_DIR,'trial_to_param.json')
        if os.path.exists(trial_to_param_path):
            with open(trial_to_param_path, "r") as f:
                data = json.load(f)
        else:
            data = {name_codified: []}

        hyperparam_codified = f"{self.model_id}_{self.lr}_{self.buffer_size}_{self.target_update_freq}_{self.gamma}_{self.eps_decay}_{self.batch_size}"
        hyperparam_codified_time = f"{timestamp}_{self.model_id}_{self.lr}_{self.buffer_size}_{self.target_update_freq}_{self.gamma}_{self.eps_decay}_{self.batch_size}"
        data[name_codified] = hyperparam_codified_time

        with open(trial_to_param_path, "w") as f:
            json.dump(data, f, indent=2)

        # Store the training configs in JSON file
        training_params = {
            'model_id':             self.model_id,
            'lr':                   self.lr,
            'gamma':                self.gamma,
            'epsilon_decay':        self.eps_decay,
            'batch_size':           self.batch_size,
            'buffer_size':          self.buffer_size,
            'target_update_freq':   self.target_update_freq
        }

        # Store training parameters in each run 
        param_path = os.path.join(OUTPUT_DIR, "param_config.json")
        with open(param_path, "w") as f:
            json.dump({"parameters": training_params}, f, indent=2)

        return name_codified, hyperparam_codified, OUTPUT_DIR
    
    def eps_greedy_policy(self, obs):      
        # TODO: epsilon can be replaced with self.epsilon, but must remember to update self.epsilon each episode
        ''' Function to take an action according to an epsilon-greedy policy and a Q-network'''
        if np.random.random() < self.eps:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(obs).unsqueeze(0)
                q_values = self.Q_net(state_tensor)
                action = q_values.argmax().item()

        return action
    
    def DDQN_train(env: gym.Env, env_test: gym.Env,
              q_network: nn.Module, target_net: nn.Module, optimizer: torch.optim.Optimizer, 
              replay_buffer,
              target_update_freq,
              gamma,
              eps_start, eps_end, eps_decay,
              episode_train,
              batch_size, 
              save_path,
              seed = 42):
    ''' Function to train a policy for a set of hyperparameters '''

    msg = "Training ended, no good model found!"

    reward_history = np.zeros(episode_train)
    epsilon = eps_start
    step_count = 0
    episode = 0
    target_network_update_count = 0

    # Control of early stopping
    consecutive_pass_count = 0           # Number of consecutive episodes where performance exceeds a threshold
    CONSECUTIVE_PASS_LIMIT = 3          # No. of consecutive episodes with higher performance than reward limit
    EPISODE_REWARD_LIMIT = 450
    best_reward = 0
    performance_crit = False
    train_terminated = False
    val_history = {}                    # Monitor which episode had a validation run, the train reward, and the validation (test) reward 

    while not train_terminated:     # Experiment level - loop through episodes
        obs, _ = env.reset(seed=seed)
        eps_rewards = 0
    
        while True:                 # Episode level - loop through steps
            action = eps_greedy_policy(env, obs, epsilon, q_network)

            # Interact with the environment
            next_obs, reward, term, trunc, _ = env.step(action)
            done = term or trunc
            replay_buffer.append((obs, action, reward, next_obs, done))
            obs = next_obs
            eps_rewards += reward
            step_count += 1

            # Train the Q-net using a batch of samples from the experience replay
            if len(replay_buffer) >= batch_size:
                batch = random.sample(replay_buffer, batch_size)
                states, actions, rewards, next_states, dones = zip(*batch)

                states = torch.FloatTensor(np.array(states))            # Shape of (BATCH_SIZE, 4). Data structure of [[p1 v1 theta1 dtheta1], [p2 v2 theta2 dtheta2], ...]
                actions = torch.LongTensor(actions).unsqueeze(1)        # Must unsqueeze to have shape (BATCH_SIZE, 1). From [a1, a2, ...] to [[a1], [a2], ... [a_BATCH_SIZE]]
                rewards = torch.FloatTensor(rewards).unsqueeze(1)
                next_states = torch.FloatTensor(np.array(next_states))
                dones = torch.FloatTensor(dones).unsqueeze(1)
                
                # Compute targets using target network Q(s',a',w_i^-)
                with torch.no_grad():
                    target_q_values = target_net(next_states)       # Find a batch of Q(s',a',w_i^-) from the batch of next_states
                    max_target_q_values = target_q_values.max(dim=1, keepdim=True)[0]
                    targets = rewards + gamma * max_target_q_values * (1 - dones)       # (1-dones) to avoid over estimating the value of terminal states. The target should only be the reward in the terminal states
            
                # Compute the current Q values for the actions taken Q(s,a,w_i)
                q_values = q_network(states).gather(1, actions)         # obtain the q_values associated to the actual action taken in each sample

                # Update the parameters of the behaviour q_network
                loss = nn.MSELoss()(q_values, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Periodically update the target network by loading the weights from the behavior network
            if step_count % target_update_freq == 0:
                target_network_update_count += 1
                target_net.load_state_dict(q_network.state_dict())

            if done:        # End of a training episode
                break

        # Decay epsilon after an episode
        epsilon = max(eps_end, epsilon - (eps_start - eps_end)/eps_decay)

        reward_history[episode] = eps_rewards
        # print(f"Episode {episode:5d}: Total reward = {eps_rewards:5.1f}, Epsilon = {epsilon:.3f}, Step count = {step_count:5d}, Target update count = {target_network_update_count:3d}")
        # print(f"Average reward {sum(reward_history[:episode])/(episode+1):.2f}")

        
        if episode % 10 == 0:                   # print progress periodically
            print(f"Episode {episode:5d}: Total reward = {eps_rewards:5.1f}, Epsilon = {epsilon:.3f}", end = "\r")

        # Early stopping condition
        if eps_rewards >= EPISODE_REWARD_LIMIT:
            test_reward = policy_eval(env_test, q_network, 100)
            val_history[episode] = [eps_rewards, test_reward]

            if test_reward > best_reward:           # Set the new best reward
                best_reward = test_reward
                save_model(q_network, optimizer, save_path)
                msg = f"Training terminated due to episode limit, best model saved at episode {episode:5d}"
            if test_reward > EPISODE_REWARD_LIMIT: 
                consecutive_pass_count += 1
            else: consecutive_pass_count = 0
        else:
            consecutive_pass_count = 0
            
        # Performance criteria - if good results for several episodes => training performance satisfied and terminate early
        if consecutive_pass_count >= CONSECUTIVE_PASS_LIMIT:
            performance_crit = True 
            msg = f"Early termination at episode {episode:5d}, desired performance reached"


        episode += 1

        # Checking for early training termination or truncation
        train_terminated = (episode >= episode_train) or (performance_crit)
    print("\n")
    return reward_history, val_history, msg


## Blah